In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, utils
from torch.autograd import Variable

torch.manual_seed(42)

# Step 1 Build data pipeline and import image dataset (split into training set and test set 5:5)

In [2]:
# data_dir is the address where you store your data data, I put it here under relative path
data_dir = "D:/zhuomian/2022_project/data"
split = 0.5
batch_size = 32
learning_rate = 0.001
num_epochs = 25

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [3]:
# Build data pre-processing
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((56,56)),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Import data into dataset
dataset = datasets.ImageFolder(root = data_dir, transform = data_transform)

# Now divide the data set into a training set and a test set
len_imgs = len(dataset.imgs)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len_imgs*split), len_imgs-int(len_imgs*split)]) 

# Final construction of the data pipeline
train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size = batch_size,shuffle = False)

# Step 2 Build the model Customized loss function and optimizer

In [4]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # conv2d Parameters nn.Conv2d(in_channels=3, out_channels=32,kernel_size=5,stride=1,padding="same")
        # MaxPool2d Parameters nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv00 = nn.Sequential(         
            nn.Conv2d(32, 48, 1, 1, padding = "same"),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 48, 3, 1, padding = "same"),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 32, 1, 1, padding = "same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        self.conv01 = nn.Sequential(         
            nn.Conv2d(64, 96, 1, 1, padding = "same"),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Conv2d(96, 96, 3, 1, padding = "same"),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Conv2d(96, 64, 1, 1, padding = "same"),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.conv02 = nn.Sequential(         
            nn.Conv2d(32, 48, 1, 1, padding = "same"),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 48, 3, 1, padding = "same"),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 32, 1, 1, padding = "same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        self.conv1 = nn.Sequential(         
            nn.Conv2d(3, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2) 
        )
        
        self.conv2 = nn.Sequential(         
            nn.Conv2d(32, 64, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        self.conv3 = nn.Sequential(         
            nn.Conv2d(64, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        self.dense = nn.Sequential(         
            nn.Conv2d(64, 32, 1, 2),
            nn.BatchNorm2d(32),
            nn.ReLU()            
        )
        
        # Fully connected layer with 21 output categories
        self.fc1 = nn.Sequential(nn.Linear(32 * 7 * 7, 128),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128, 21))
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        
        identity1 = x
        x = self.conv01(x)
        x = identity1 + x
        #
        identity1 = self.dense(x)
        #
        x = self.conv3(x)
        
        identity2 = x
        x = self.conv02(x)
        x = identity2 + x
        
        x = identity1 + x
        # Flattening treatment x.view
        x = x.view(x.size(0), -1)       
        x = self.fc1(x)
        output = self.fc2(x)
        return output    # return x for visualization

In [5]:
model = CNN_model()
model.to(device)
# Choose whether to print the model or not, here it will not be printed, just comment
print(model)

# Select loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate, betas=(0.9, 0.999),
                       eps=1e-07, amsgrad=False)

CNN_model(
  (conv00): Sequential(
    (0): Conv2d(32, 48, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (conv01): Sequential(
    (0): Conv2d(64, 96, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(96, 64, kernel_size=(1, 1), str

# Step 3 Pass in the model to start training and testing

In [6]:
# Train the model and record the accuracy and loss values for each epoch during training
def train(num_epochs, model, loaders):
    model.train()   
    
    # The first stores the correct rate and the second stores the loss value
    Accy_list = []
    Loss_list = []
    
    total_step = len(loaders)    
    for epoch in range(num_epochs):
        correct = 0
        L = 0
        for i, (images, labels) in enumerate(loaders):
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            # Clear the previous gradient 
            optimizer.zero_grad()           
            
            # Backpropagate and update the parameters
            loss.backward()               
            optimizer.step()                
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum()
            L += loss.item()
            if (i+1) % 9 == 0:
                print ('Epoch: [{}/{}]\t|\tStep: [{}/{}]\t|\tLoss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                
        accuracy = 100 * correct / len(train_set)
        print(accuracy.item())
        print("epoch: {:02d} | Accuracy: {:.2f} Loss: {:.4f}\n"
              .format(epoch + 1, accuracy, L))
        
        # Add the correct and lost values for this epoch to the corresponding list
        Accy_list.append(accuracy.item())
        Loss_list.append(L)
              
    return Accy_list, Loss_list


Accy_list, Loss_list = train(num_epochs, model, train_loader)

Epoch: [1/25]	|	Step: [9/27]	|	Loss: 3.0491
Epoch: [1/25]	|	Step: [18/27]	|	Loss: 3.0271
Epoch: [1/25]	|	Step: [27/27]	|	Loss: 3.0307
5.945303440093994
epoch: 01 | Accuracy: 5.95 Loss: 82.5760

Epoch: [2/25]	|	Step: [9/27]	|	Loss: 2.7838
Epoch: [2/25]	|	Step: [18/27]	|	Loss: 2.7735
Epoch: [2/25]	|	Step: [27/27]	|	Loss: 2.4575
17.954814910888672
epoch: 02 | Accuracy: 17.95 Loss: 74.6046

Epoch: [3/25]	|	Step: [9/27]	|	Loss: 2.3978
Epoch: [3/25]	|	Step: [18/27]	|	Loss: 2.2768
Epoch: [3/25]	|	Step: [27/27]	|	Loss: 2.5146
29.250892639160156
epoch: 03 | Accuracy: 29.25 Loss: 64.2070

Epoch: [4/25]	|	Step: [9/27]	|	Loss: 1.6868
Epoch: [4/25]	|	Step: [18/27]	|	Loss: 1.7660
Epoch: [4/25]	|	Step: [27/27]	|	Loss: 1.5011
44.82758712768555
epoch: 04 | Accuracy: 44.83 Loss: 48.3145

Epoch: [5/25]	|	Step: [9/27]	|	Loss: 1.4935
Epoch: [5/25]	|	Step: [18/27]	|	Loss: 1.2865
Epoch: [5/25]	|	Step: [27/27]	|	Loss: 1.7751
60.998809814453125
epoch: 05 | Accuracy: 61.00 Loss: 34.7399

Epoch: [6/25]	|	Step: [

In [7]:
print(Accy_list)

[5.945303440093994, 17.954814910888672, 29.250892639160156, 44.82758712768555, 60.998809814453125, 75.98097229003906, 81.45065307617188, 89.77407836914062, 93.81688690185547, 97.38406372070312, 99.04875183105469, 99.28656005859375, 99.88109588623047, 98.92984771728516, 100.0, 100.0, 100.0, 99.88109588623047, 100.0, 100.0, 99.7621841430664, 100.0, 100.0, 100.0, 100.0]


In [8]:
print(Loss_list)

[82.57595610618591, 74.6046347618103, 64.20697402954102, 48.314494371414185, 34.73991322517395, 22.68424701690674, 16.565885588526726, 11.13362717628479, 7.274530507624149, 3.661134459078312, 2.6363786794245243, 2.0224409848451614, 1.4268008125945926, 1.8484936840832233, 0.7699930025264621, 0.3480021804571152, 0.2657509045675397, 0.306058204267174, 0.19224251667037606, 0.166781775187701, 0.5868758331052959, 0.2876351922750473, 0.18293593428097665, 0.18935117172077298, 0.14978503878228366]


In [9]:
# Test models
def test(model, loaders):
    model.eval()
    with torch.no_grad():
        correct = 0
        L = 0
        for images, labels in loaders:
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            # Get the loss value and add it up
            loss = loss_func(outputs, labels)
            L += loss.item()
            
             # Get the correct number of predicted samples
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).float().sum()
            
    accuracy = (100 * correct / len(test_set))
    print("Test data | Accuracy: {:.2f} %, Loss: {:.4f} ".format(accuracy, L))
    
    return accuracy, L

Test_acc, Test_loss = test(model, test_loader)

Test data | Accuracy: 85.14 %, Loss: 14.3850 
